In [ ]:
#https://www.data.go.kr/data/15000563/openapi.do

In [ ]:
import requests
import xmltodict

# MySQL에 약국 데이터 저장
def save_to_mysql(data):
    import pymysql
    import configparser

    # ConfigParser 초기화
    config = configparser.ConfigParser()

    # keys.config 파일 읽기
    config.read('../keys.config')

    import pymysql
    connection = pymysql.connect(host = config['DB_INFO']['host'],
                        port = 3306,
                        user = config['DB_INFO']['id'],
                        passwd = config['DB_INFO']['password'],
                        db = config['DB_INFO']['db'],
                        charset = 'utf8') # 한글 깨짐 방지
        
    cursor = connection.cursor(pymysql.cursors.DictCursor)

    
    # 새 데이터 삽입
    insert_query = """
        INSERT INTO pharmacy_data (
            dutyAddr, dutyMapimg, dutyName, dutyTel1, 
            dutyTime1s, dutyTime1c, dutyTime2s, dutyTime2c,
            dutyTime3s, dutyTime3c, dutyTime4s, dutyTime4c, dutyTime5s, dutyTime5c, 
            dutyTime6s, dutyTime6c, dutyTime7s, dutyTime7c, dutyTime8s, dutyTime8c, 
            postCdn1, postCdn2, wgs84Lat, wgs84Lon, dutyEtc
        ) 
        VALUES (
            %s, %s, %s, %s, 
            %s, %s, %s, %s,
            %s, %s, %s, %s, %s, %s, 
            %s, %s, %s, %s, %s, %s, 
            %s, %s, %s, %s, %s
        )
    """

    cursor.executemany(insert_query, data)
    connection.commit()
    cursor.close()
    connection.close()